In [8]:
from radd import neuro, build
from radd.toolbox import analyze

In [ ]:
analyze.assess_fit()

In [38]:
m.avg_wts

array([ 1.44875,  0.92543,  0.84907,  0.82282,  1.08764,  1.49848,
        0.32548,  1.22685,  5.27358,  3.70829,  0.90674,  0.09328,
        0.52865,  1.22075,  1.06581,  0.28346])

In [133]:
vfit=pd.read_csv("./xpro_v_bootinfo.csv")

In [204]:
trfit=pd.read_csv("./xpro_tr_bootinfo.csv")

In [209]:
inits = m.__get_optimized_params__()

In [213]:
m.simulate()

array([ 0.93457,  0.73709,  0.47719,  0.2461 ,  0.07923,  0.05282,
        0.48514,  0.5047 ,  0.51599,  0.5288 ,  0.5448 ,  0.51899,
        0.53199,  0.5407 ,  0.54699,  0.5527 ])

In [211]:
inits

{'a': 0.331967,
 'tr': array([ 0.37042,  0.3537 ,  0.33699,  0.3207 ,  0.29899,  0.2908 ]),
 'v': 1.377375,
 'xb': 1.996667}

In [298]:
inits = trfit.iloc[40]['a':'xb'].to_dict()

In [135]:
inits = vfit.iloc[1]['a':'xb'].to_dict()

In [280]:
inits = trfit[trfit.AIC == trfit.AIC.min()].loc[:, 'a':'xb'].T.to_dict()[59]

In [299]:
trc = ['_'.join(['tr', str(p) ]) for p in np.arange(0, 120, 20)]
inits['tr']=array([inits[xc] for xc in trc])      

In [277]:
trc = ['_'.join(['tr', str(p) ]) for p in np.arange(0, 120, 20)]
inits['tr']=array([inits[xc] for xc in trc])      

KeyError: 'tr_0'

In [423]:
m = build.Model(data=prodf_store.prodata, kind='xpro', depends_on={'tr':'pGo'}, inits=inits)
m.inits=m.__get_optimized_params__()

In [424]:
b = neuro.BOLD(m)

In [420]:
b.make_bold_dfs()

In [425]:
b.simulate_bold()

In [308]:
b.onset

array([ 0.29212,  0.29212,  0.29213,  0.29199,  0.29212,  0.29212], dtype=float32)

In [ ]:
.4/(1+(1.5*(np.arange(100))))

In [297]:
b.p

{'a': array([ 0.06721,  0.06721,  0.06721,  0.06721,  0.06721,  0.06721]),
 'tr': array([ 0.51771,  0.50482,  0.49169,  0.47993,  0.45334,  0.44865], dtype=float32),
 'tr_0': 0.51771429535300006,
 'tr_100': 0.44864876317599994,
 'tr_20': 0.50481884447800007,
 'tr_40': 0.49169071394500002,
 'tr_60': 0.47992701621699996,
 'tr_80': 0.453343268824,
 'v': array([ 0.95975,  0.95975,  0.95975,  0.95975,  0.95975,  0.95975]),
 'xb': array([ 1.4676,  1.4676,  1.4676,  1.4676,  1.4676,  1.4676])}

In [156]:
tri={'tr_80': 0.28484007545292894, 'a': array([ 0.32416,  0.32416,  0.32416,  0.32416,  0.32416,  0.32416]), 'xb': array([ 1.,  1.,  1.,  1.,  1.,  1.]), 'tr_0': 0.36461228199213791, 'tr': array([ 0.36461,  0.34395,  0.3215 ,  0.31063,  0.28484,  0.27668]), 'tr_60': 0.31063347462675678, 'tr_20': 0.34395352442654509, 'tr_40': 0.32150306384276961, 'v': array([ 1.37767,  1.37767,  1.37767,  1.37767,  1.37767,  1.37767]), 'tr_100': 0.27668469302300397}

In [209]:
popt={'tr_80': 0.26747523622695923, 'a': array([ 0.4748,  0.4748,  0.4748,  0.4748,  0.4748,  0.4748]), 'xb': array([ 1.5,  1.5,  1.5,  1.5,  1.5,  1.5]), 'tr_0': 0.32712324490086442, 'tr': array([ 0.32712,  0.31299,  0.29496,  0.28653,  0.26748,  0.26579]), 'tr_60': 0.28653386965252559, 'tr_20': 0.31298907976434909, 'tr_40': 0.29496087121742431, 'v': array([ 1.6961,  1.6961,  1.6961,  1.6961,  1.6961,  1.6961]), 'tr_100': 0.26578653965638499}


In [434]:
b.p=popt
b.simulate_bold()
b.make_bold_dfs()

In [435]:
b.plot_traces(style='HL')

In [436]:
b.plot_means()

In [388]:
gohi = b.go_traces[-1]
#gohi.iloc[-100:, :30]

In [99]:
nohi = b.ng_traces[-1]

In [219]:
dt=b.dt
si=.01
dx=np.sqrt(si*dt)
Pg, Tg, xtb, t = __update_go_process__(m.inits)
DVg = xtb[:,None]*np.cumsum(np.where((rs((6,1,Tg.max())).T < Pg), dx,-dx).T, axis=2)
plt.plot(t, DVg[0,0])
def __update_go_process__(p):
      """ update Pg (probability of DVg +dx) and Tg (num go process timepoints)
      for go process and get get dynamic bias signal if 'x' model
      """
      p['xb'] = np.ones(6)*5
      dt=b.dt
      si=.01
      dx=np.sqrt(si*dt)
      Pg = 0.5*(1 + p['v']*dx/si)
      Tg = np.ceil((.55-p['tr'])/dt).astype(int)
      t = np.cumsum([dt]*Tg.max())
      xtb = np.cosh(p['xb'][:,None]*t)
      return Pg, Tg, xtb, t

In [110]:
gy.dropna(inplace=True)

In [114]:
ny = nohi.iloc[:, np.argmin(nohi.iloc[-1, :])].values
gy = gohi.iloc[:, 0].dropna().values

In [117]:
xg = b.onset[-1]+np.arange(len(gy))*b.dt
xn = b.onset[-1]+np.arange(len(ny))*b.dt

In [430]:
b.simulate_bold(ntrials=30)

In [431]:
b.plot_means()

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(12,5))

In [3]:
ai={'a': array([ 0.37922,  0.34356,  0.30503,  0.28076,  0.24567,  0.23642]), 'a_20': 0.34355607211860251, 'a_0': 0.37922233730662624, 'a_100': 0.23642099108705172, 'tr': array([ 0.29971,  0.29971,  0.29971,  0.29971,  0.29971,  0.29971]), 'a_80': 0.24567196344547787, 'xb': array([ 0.91365,  0.91365,  0.91365,  0.91365,  0.91365,  0.91365]), 'a_60': 0.28075925861695922, 'a_40': 0.30503138948470693, 'v': array([ 1.12735,  1.12735,  1.12735,  1.12735,  1.12735,  1.12735])}

In [4]:
vi={'a': array([ 0.48723,  0.48723,  0.48723,  0.48723,  0.48723,  0.48723]), 'v_0': 1.4110558806716156, 'xb': array([ 1.56347,  1.56347,  1.56347,  1.56347,  1.56347,  1.56347]), 'v': array([ 1.41106,  1.56252,  1.68275,  1.76071,  1.87995,  1.92475]), 'tr': array([ 0.29213,  0.29213,  0.29213,  0.29213,  0.29213,  0.29213]), 'v_100': 1.9247546481573701, 'v_60': 1.7607100143415964, 'v_80': 1.8799486738431181, 'v_20': 1.562521284952787, 'v_40': 1.6827500266974127}

In [150]:

p={'tr_80': 0.084728912835387152, 'a': array([ 0.62868,  0.62868,  0.62868,  0.62868,  0.62868,  0.62868]), 'xb': array([ 0.64083,  0.64083,  0.64083,  0.64083,  0.64083,  0.64083]), 'tr_0': 0.18202827534003851, 'tr': array([ 0.18203,  0.16131,  0.13355,  0.11741,  0.08473,  0.07697]), 'tr_60': 0.11740854432703671, 'tr_20': 0.16131161484116485, 'tr_40': 0.13354796995159479, 'v': array([ 1.42112,  1.42112,  1.42112,  1.42112,  1.42112,  1.42112]), 'tr_100': 0.076965157557332742}

In [123]:
temporal_dynamics = lambda p, t: np.cosh(p['xb'][:,None]*t)

In [128]:
xtb

array([[ 1.     ,  1.     ,  1.     , ...,  1.04619,  1.04629,  1.04639],
       [ 1.     ,  1.     ,  1.     , ...,  1.04619,  1.04629,  1.04639],
       [ 1.     ,  1.     ,  1.     , ...,  1.04619,  1.04629,  1.04639],
       [ 1.     ,  1.     ,  1.     , ...,  1.04619,  1.04629,  1.04639],
       [ 1.     ,  1.     ,  1.     , ...,  1.04619,  1.04629,  1.04639],
       [ 1.     ,  1.     ,  1.     , ...,  1.04619,  1.04629,  1.04639]])

In [130]:
diff

[211, 169, 114, 81, 16, 0]

In [143]:
x= [np.append(np.ones(diff[i]), xtb[i][:-diff[i]]) for tg in Tg]

In [145]:
#Pg = 0.5*(1 + p['v']*self.dx/self.si)

Tg = np.ceil((.55-p['tr'])/.0005).astype(int)
t = np.cumsum([.0005]*Tg.max())
xtb = temporal_dynamics(p, t)
diff = [Tg.max()-tg for tg in Tg]
xtb = np.vstack([np.append(np.ones(diff[i]), xtb[i][:tg]) for i, tg in enumerate(Tg)])
#diff = [Tg.max()-tg for tg in Tg]
#[np.append(np.ones(diff[i]), [.0005]*Tg[i]) for i in range(4)]#

In [148]:
len(xtb[-1])

947

In [149]:
diff[-1]

0

In [233]:
vi={'a': array([ 0.1,  0.1,  0.1,  0.1,  0.1,  0.1]), 'v_0': 0.59438867299421838, 'xb': array([ 3.61495,  3.61495,  3.61495,  3.61495,  3.61495,  3.61495]), 'v': array([ 0.59439,  0.81921,  1.03934,  1.22616,  1.45434,  1.54175]), 'tr': array([ 0.47004,  0.47004,  0.47004,  0.47004,  0.47004,  0.47004]), 'v_100': 1.5417474028490836, 'v_60': 1.2261613000758063, 'v_80': 1.4543394294648317, 'v_20': 0.81921386004979269, 'v_40': 1.0393395139558887}
vi={'a': array([ 0.51141,  0.51141,  0.51141,  0.51141,  0.51141,  0.51141]), 'v_0': 1.2206296380937256, 'xb': array([ 3.80715,  3.80715,  3.80715,  3.80715,  3.80715,  3.80715]), 'v': array([ 1.22063,  1.36224,  1.49704,  1.60176,  1.72381,  1.76406]), 'tr': array([ 0.32001,  0.32001,  0.32001,  0.32001,  0.32001,  0.32001]), 'v_100': 1.7640615823592622, 'v_60': 1.6017636718723776, 'v_80': 1.7238067015995366, 'v_20': 1.3622358968043313, 'v_40': 1.4970380849011338}


In [210]:
tri = {'a':0.4748, 'tr': array([0.32712,  0.31299, 0.29496, 0.28653, 0.26748, 0.26579]), 'v':1.6961, 'xb':1.5}
vi={'a': array([ 0.37609,  0.37609,  0.37609,  0.37609,  0.37609,  0.37609]), 'v_0': 1.0643536982493442, 'xb': array([ 3.9265,  3.9265,  3.9265,  3.9265,  3.9265,  3.9265]), 'v': array([ 1.06435,  1.21267,  1.34857,  1.45434,  1.59614,  1.63391], dtype=float32), 'tr': array([ 0.35126,  0.35126,  0.35126,  0.35126,  0.35126,  0.35126]), 'v_100': 1.6339055193390075, 'v_60': 1.4543434473306287, 'v_80': 1.5961424299208138, 'v_20': 1.2126660918682359, 'v_40': 1.3485718698710036}

In [234]:
dp=['v', 'tr', 'a']
inits=[vi, tri, ai]
models = [build.Model(data=pro, kind='xpro', depends_on={pkey:'pGo'}, inits=inits[i]) for i, pkey in enumerate(dp)]
neuro_models = [neuro.BOLD(m) for m in models]

In [235]:
for nm in neuro_models:
      nm.simulate_bold(ntrials=100)  

In [236]:
vb = neuro_models[0]
trb = neuro_models[1]
ab = neuro_models[2]

In [226]:
godf = trb.bold_mag[trb.bold_mag.choice=='go']

In [231]:
godf.columns

Index([u'choice', u'cond', u'csum'], dtype='object')

In [232]:
from scipy.stats import ttest_rel
low = godf[godf.cond==60].csum.values
high = godf[godf.cond==100].csum.values[:len(low)]
ttest_rel(low, high)

(-5.8412589155062369, 8.281969849979086e-09)

In [242]:
vb.simulate_bold(ntrials=100)
vb.plot_means()
vb.plot_traces()

In [ ]:
b.tb=.56

In [100]:
f, axes = plt.subplots(3, 2, figsize=(12,15))
lx=False

for i, nm in enumerate(neuro_models):
      if i==(len(neuro_models)-1):
            lx=True
      nm.plot_traces(ax=axes[i][0], label_x=lx)
      nm.plot_means(ax=axes[i][1], label_x=lx)
#plt.savefig('F7.png',dpi=700, save=True)
plt.savefig('F7.svg', rasterized=True)